In [1]:
%load_ext autoreload
%autoreload 2
from notebook import *
# if get something about NUMEXPR_MAX_THREADS being set incorrectly, don't worry.  It's not a problem.

# Code Example: Linked list v.s. array

In [2]:
compare([do_render_code("linked_list/linked_list.c", show="depth_of_array"),do_render_code("linked_list/linked_list.c", show="depth_of_list")])

In [3]:
! make -C linked_list clean ; make -C linked_list
! echo "size,list,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses" > stats.csv
! echo -n "4096,0," >> stats.csv
! linked_list/linked_list 4096 0
! echo -n "4096,1," >> stats.csv
! linked_list/linked_list 4096 1

make: Entering directory '/nfshome/htseng/courses/CSE142/demo/OoO_programming/linked_list'
rm -f linked_list linked_list.s
make: Leaving directory '/nfshome/htseng/courses/CSE142/demo/OoO_programming/linked_list'
make: Entering directory '/nfshome/htseng/courses/CSE142/demo/OoO_programming/linked_list'
cc -O3 -Wno-implicit-function-declaration -Wno-format-zero-length -DHAVE_LINUX_PERF_EVENT_H perfstats.c  linked_list.c -o linked_list
cc -S -O3 -Wno-implicit-function-declaration -Wno-format-zero-length -DHAVE_LINUX_PERF_EVENT_H linked_list.c 
make: Leaving directory '/nfshome/htseng/courses/CSE142/demo/OoO_programming/linked_list'
depth: 4096
depth: 4096


In [4]:
display_df_mono(render_csv("stats.csv"))

,size,list,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses
0,4096,0,29749,12146,0.408283,0.493990,0.000006,nan,0,0,5135,37
1,4096,1,21548,45423,2.107991,0.176122,0.000008,nan,0,0,5138,52


In [5]:
compare([do_render_code("linked_list/linked_list.s", show=["depth_of_array:",".LFE40:"]),do_render_code("linked_list/linked_list.s", show=["depth_of_list",".LFE39:"])])

# Code example: popcount

Given a 64-bit integer number, find the number of 1s in its binary representation.
Example 1:
Input: 59487
Output: 9

Explanation: 9487’s binary representation is 0b10110010100001111

In [6]:
render_code("popcounts/popcount.cpp", show=["#else //A", "#endif"])

// popcounts/popcount.cpp:80-90 (11 lines)
#else //A
inline int popcount(uint64_t x){
	int c=0;
	while(x) //while any bits are set not sure how you are getting the value
    {
       c += x & 1;
       x = x >> 1;
    }
    return c;
}
#endif

In [7]:
render_code("popcounts/popcount.cpp", show=["#ifdef B", "#else"])

// popcounts/popcount.cpp:25-41 (17 lines)
#ifdef B
inline int popcount(uint64_t x) {
     int c = 0;
	 while(x) //while any bits are set not sure how you are getting the value
     {
       c += x & 1;
       x = x >> 1;
       c += x & 1;
       x = x >> 1;
       c += x & 1;
       x = x >> 1;
       c += x & 1;
       x = x >> 1;
     }
     return c;
}
#else

In [8]:
render_code("popcounts/popcount.cpp", show=["#ifdef C", "#else"])

// popcounts/popcount.cpp:13-24 (12 lines)
#ifdef C
inline int popcount(uint64_t x) {
     int c = 0;
     int table[16] = {0, 1, 1, 2, 1, 2, 2, 3, 1, 2, 2, 3, 2, 3, 3, 4};
     while(x) {
	       c += table[(x & 0xF)];
	       x = x >> 4;
     }
     return c;
}

#else

In [9]:
render_code("popcounts/popcount.cpp", show=["#ifdef D", "#else"])

// popcounts/popcount.cpp:42-52 (11 lines)
#ifdef D
inline int popcount(uint64_t x) {
     int c = 0;
     int table[16] = {0, 1, 1, 2, 1, 2, 2, 3, 1, 2, 2, 3, 2, 3, 3, 4};
     for (uint64_t i = 0; i < 16; i++) {
	       c += table[(x & 0xF)];
	       x = x >> 4;
     }
     return c;
}
#else

In [10]:
render_code("popcounts/popcount.cpp", show=["#ifdef E", "#else"])

// popcounts/popcount.cpp:53-80 (28 lines)
#ifdef E
inline int popcount(uint64_t x) {
     int c = 0;
     for (uint64_t i = 0; i < 16; i++) {
         switch((x & 0xF))
         {
             case 1: c+=1; break;
             case 2: c+=1; break;
             case 3: c+=2; break;
             case 4: c+=1; break;
             case 5: c+=2; break;
             case 6: c+=2; break;
             case 7: c+=3; break;
             case 8: c+=1; break;
             case 9: c+=2; break;
             case 10: c+=2; break;
             case 11: c+=3; break;
             case 12: c+=2; break;
             case 13: c+=3; break;
             case 14: c+=3; break;
             case 15: c+=4; break;
             default: break;
         }
         x = x >> 4;
     }
     return c;
}
#else //A

## Who has the best performance?

In [11]:
render_code("popcounts/popcount.cpp", show="main")

// popcounts/popcount.cpp:107-120 (14 lines)
int main(int argc, char *argv[]) {

     uint64_t key = 0xdeadbeef;

     int count = 1000000000;
     uint64_t sum = 0;
     
     for (int i=0; i < count; i++)
     { 
	   sum += popcount (RandLFSR(key)); 
     }
     printf("Result: %lu\n", sum);
     return sum;
}

In [ ]:
! make -C popcounts clean; make -C popcounts
! echo "Version A"; time ./popcounts/popcount_A
! echo "Version B";time ./popcounts/popcount_B
! echo "Version C";time ./popcounts/popcount_C
! echo "Version D";time ./popcounts/popcount_D
! echo "Version E";time ./popcounts/popcount_E

make: Entering directory '/nfshome/htseng/courses/CSE142/demo/OoO_programming/popcounts'
rm -f popcount_A popcount_B popcount_C popcount_D popcount_SSE42 popcount_E
make: Leaving directory '/nfshome/htseng/courses/CSE142/demo/OoO_programming/popcounts'
make: Entering directory '/nfshome/htseng/courses/CSE142/demo/OoO_programming/popcounts'
g++ -O3 popcount.cpp -o popcount_A
g++ -S -O3 popcount.cpp -o A.s
g++ -O3 -DB popcount.cpp -o popcount_B
g++ -S -O3 -DB popcount.cpp -o B.s
g++ -O3 -DC popcount.cpp -o popcount_C
g++ -S -O3 -DC popcount.cpp -o C.s
g++ -O3 -DD popcount.cpp -o popcount_D
g++ -S -O3 -DD popcount.cpp -o D.s
g++ -O3 -m64 -msse4.2 -DSSE42 popcount.cpp -o popcount_SSE42
g++ -O3 -DE popcount.cpp -o popcount_E
g++ -S -O3 -DE popcount.cpp -o E.s
make: Leaving directory '/nfshome/htseng/courses/CSE142/demo/OoO_programming/popcounts'
Version A
Result: 32000010811

real	0m22.231s
user	0m22.225s
sys	0m0.004s
Version B
Result: 32000010811

real	0m12.322s
user	0m12.320s
sys	0m0.000s

In [22]:
! perf stat ./popcounts/popcount_A

Result: 32000010811

 Performance counter stats for './popcounts/popcount_A':

         22,198.74 msec task-clock                #    1.000 CPUs utilized          
                78      context-switches          #    3.514 /sec                   
                 3      cpu-migrations            #    0.135 /sec                   
               122      page-faults               #    5.496 /sec                   
   114,891,715,231      cycles                    #    5.176 GHz                    
   332,043,767,058      instructions              #    2.89  insn per cycle         
    65,007,933,051      branches                  #    2.928 G/sec                  
       724,687,423      branch-misses             #    1.11% of all branches        

      22.200576455 seconds time elapsed

      22.198996000 seconds user
       0.000000000 seconds sys




In [23]:
! perf stat ./popcounts/popcount_B

Result: 32000010811

 Performance counter stats for './popcounts/popcount_B':

         12,305.19 msec task-clock                #    1.000 CPUs utilized          
                48      context-switches          #    3.901 /sec                   
                 3      cpu-migrations            #    0.244 /sec                   
               126      page-faults               #   10.240 /sec                   
    63,682,315,849      cycles                    #    5.175 GHz                    
   287,892,407,376      instructions              #    4.52  insn per cycle         
    17,937,966,486      branches                  #    1.458 G/sec                  
        63,536,893      branch-misses             #    0.35% of all branches        

      12.306358457 seconds time elapsed

      12.305376000 seconds user
       0.000000000 seconds sys




In [33]:
! perf stat ./popcounts/popcount_C

Result: 32000010811

 Performance counter stats for './popcounts/popcount_C':

          5,022.97 msec task-clock                #    1.000 CPUs utilized          
                14      context-switches          #    2.787 /sec                   
                 1      cpu-migrations            #    0.199 /sec                   
               125      page-faults               #   24.886 /sec                   
    25,984,158,973      cycles                    #    5.173 GHz                    
   102,679,397,712      instructions              #    3.95  insn per cycle         
    17,935,624,177      branches                  #    3.571 G/sec                  
        67,091,007      branch-misses             #    0.37% of all branches        

       5.024102191 seconds time elapsed

       5.019143000 seconds user
       0.003999000 seconds sys




In [30]:
! perf stat ./popcounts/popcount_D

Result: 32000010811

 Performance counter stats for './popcounts/popcount_D':

          3,742.44 msec task-clock                #    1.000 CPUs utilized          
                15      context-switches          #    4.008 /sec                   
                 1      cpu-migrations            #    0.267 /sec                   
               123      page-faults               #   32.866 /sec                   
    19,324,958,594      cycles                    #    5.164 GHz                    
    80,010,186,103      instructions              #    4.14  insn per cycle         
     1,001,823,401      branches                  #  267.693 M/sec                  
            21,566      branch-misses             #    0.00% of all branches        

       3.743956669 seconds time elapsed

       3.742616000 seconds user
       0.000000000 seconds sys




In [31]:
! perf stat ./popcounts/popcount_E

Result: 32000010811

 Performance counter stats for './popcounts/popcount_E':

         54,684.17 msec task-clock                #    1.000 CPUs utilized          
               174      context-switches          #    3.182 /sec                   
                16      cpu-migrations            #    0.293 /sec                   
               124      page-faults               #    2.268 /sec                   
   282,155,522,005      cycles                    #    5.160 GHz                    
   173,103,407,446      instructions              #    0.61  insn per cycle         
    44,018,637,471      branches                  #  804.961 M/sec                  
     8,174,235,653      branch-misses             #   18.57% of all branches        

      54.685822288 seconds time elapsed

      54.684550000 seconds user
       0.000000000 seconds sys




### Hardware acceleration!

In [34]:
! perf stat ./popcounts/popcount_SSE42

Result: 32000010811

 Performance counter stats for './popcounts/popcount_SSE42':

          1,577.55 msec task-clock                #    0.999 CPUs utilized          
                 5      context-switches          #    3.169 /sec                   
                 1      cpu-migrations            #    0.634 /sec                   
               125      page-faults               #   79.237 /sec                   
     8,161,753,820      cycles                    #    5.174 GHz                    
    22,006,517,918      instructions              #    2.70  insn per cycle         
     1,001,147,388      branches                  #  634.622 M/sec                  
            19,396      branch-misses             #    0.00% of all branches        

       1.578557655 seconds time elapsed

       1.577690000 seconds user
       0.000000000 seconds sys




## Do we need to use bitwise operators?


In [ ]:
compare([do_render_code("instructions/arithmetics_2.c",show=["#ifdef A","#else"]),do_render_code("instructions/arithmetics_2.c",show=["#else","#endif"])])

In [ ]:
! make -C instructions clean; make -C instructions
! echo "Version A"
! instructions/arithmetics_2_A
! instructions/arithmetics_2_A
! instructions/arithmetics_2_A
! instructions/arithmetics_2_A
! instructions/arithmetics_2_A
! echo "Version B"
! instructions/arithmetics_2_B
! instructions/arithmetics_2_B
! instructions/arithmetics_2_B
! instructions/arithmetics_2_B
! instructions/arithmetics_2_B

In [ ]:
! perf stat instructions/arithmetics_2_A
! perf stat instructions/arithmetics_2_B

Compiler can do fair amount of "obvious" optimizations.